In [ ]:
import fitz
from tqdm import tqdm
        

In [ ]:
# path = "./textbooks"
pdf_file = "./textbook.pdf"
doc = fitz.open(pdf_file)

In [ ]:
index_str = ""
for i in range(6, 16):
    index_str += doc[i].get_text()

In [ ]:
from collections import defaultdict

index_ = index_str.split('\n')
chapters = defaultdict(dict)

for i in range(1,29):
    chap = f'CHAPTER {i}'
    sections = {}
    in_sections = {}
    for j in range(len(index_)):
        if index_[j] == chap:
            chapters[chap.replace(' ', '_')]['name'] = index_[j+1].replace(' ', '_')
            chapters[chap.replace(' ', '_')]['start_page'] = index_[j+2]
        
        if index_[j].startswith(f'{i}.'):
            sections[(index_[j].replace(' ', '_').replace('10', 'X').replace('11', 'Y'))] = index_[j+1]
            in_sections[index_[j]] = int(index_[j+1]) - int(chapters[chap.replace(' ', '_')]['start_page']) + 1

        if index_[j] == "Key Terms":
            chapters[chap.replace(' ', '_')]['end_page'] = int(index_[j+1])-1
                
        if index_[j] == f'CHAPTER {i+1}':
            chapters[chap.replace(' ', '_')]['sections'] = sections
            chapters[chap.replace(' ', '_')]['in_sections'] = in_sections
            break

chapters['CHAPTER_28']['sections'] = sections
chapters['CHAPTER_28']['in_sections'] = sections
        

In [ ]:
doc[30].get_text().replace('\n', ' ').replace('Access for free at openstax.org', '')

In [ ]:
chapters

In [ ]:
import json

out_file = open("page_info.json", "w")
json.dump(chapters, out_file, indent=6)
out_file.close()

In [ ]:
import os

doc = doc
index_end = 16

os.mkdir(f'./textbooks/textbook')

for chapter in chapters.keys():
    
    chapter_name = chapters[chapter]['name']
    sections = chapters[chapter]['sections']
    page = int(chapters[chapter]['start_page']) + index_end - 1
    sorted_index = sorted(sections)

    os.mkdir(f'./textbooks/textbook/{chapter}_{chapter_name}')

    text = ""
    print(sorted_index)
    if chapters[chapter]['start_page'] in sections.values():
        text += doc[page].get_text()
    else:
        while str((page - index_end + 1)) not in sections.values():
            text += doc[page].get_text()
            page += 1

    intro_file = open(f'./textbooks/textbook/{chapter}_{chapter_name}/introduction.txt', 'w', encoding='utf-8')
    intro_file.write(text)
    

    for i in range(len(sorted_index)):
        text = ""
        if i == len(sorted_index) - 1:
            end_page = int(chapters[chapter]['end_page']) + index_end - 1
        else:
            end_page = int(sections[sorted_index[i+1]]) + index_end - 1

        while page != end_page:
            text += doc[page].get_text()
            page += 1
        
        section_content_file = open(f'./textbooks/textbook/{chapter}_{chapter_name}/{sorted_index[i]}.txt', 'w', encoding='utf-8')
        section_content_file.write(text)